<a href="https://colab.research.google.com/github/Nekhaenko/BigData1/blob/master/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colab!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import autogen  # Импорт библиотеки AutoGen для создания многоагентных систем

In [2]:
# Конфигурация для локального сервера Ollama, определяющая параметры модели
config_list = [
    {
        "model": "codellama:7b",  # Используем модель CodeLLaMA с 7 миллиардами параметров для генерации кода
        "api_type": "ollama",  # Интерфейс API для подключения к Ollama
        "client_host": "http://localhost:11434",  # Локальный адрес сервера Ollama
        "num_predict": 1024,  # Максимальное количество токенов в ответе
        "temperature": 0.1,  # Низкая температура для детерминированных ответов
        "top_p": 0.95,  # Топ-95% вероятных токенов для генерации
        "stream": False  # Отключение потоковой передачи ответа
    }
]

# Определение агента: Менеджер продукта
product_manager = autogen.ConversableAgent(
    name="product_manager",  # Уникальное имя агента
    system_message="""Вы продукт менеджер, который принимает идею и описывает план действий,
которые должен реализовать software_engineer""",  # Инструкция: формировать план для разработчика
    llm_config={"config_list": config_list},  # Конфигурация модели
    human_input_mode="NEVER",  # Полностью автономная работа без вмешательства человека
)

In [3]:
# Определение агента: Разработчик
software_engineer = autogen.ConversableAgent(
    name="software_engineer",  # Уникальное имя агента
    system_message="Вы разработчик, который принимает план действий от product_manager и пишите код",  # Инструкция: писать код по плану
    llm_config={"config_list": config_list},  # Конфигурация модели
    human_input_mode="NEVER",  # Автономная работа
)

# Определение агента: Специалист по тестированию
qa_engineer = autogen.ConversableAgent(
    name="qa_engineer",  # Уникальное имя агента
    system_message="Вы специалист по тестированию, вы принимаете код от software_engineer и пишите unit tests",  # Инструкция: создавать юнит-тесты
    llm_config={"config_list": config_list},  # Конфигурация модели
    human_input_mode="NEVER",  # Автономная работа
)

# Определение агента: DevOps-инженер
devops_engineer = autogen.ConversableAgent(
    name="devops_engineer",  # Уникальное имя агента
    system_message="""Вы devops инженер, который учитывает контекст от product_manager, software_engineer, qa_engineer
и реализовываете CI/CD""",  # Инструкция: разрабатывать CI/CD на основе работы других агентов
    llm_config={"config_list": config_list},  # Конфигурация модели
    human_input_mode="NEVER",  # Автономная работа
)

# Запуск последовательных чатов, инициируемых product_manager
chat_results = product_manager.initiate_chats(
    [
        {
            "recipient": software_engineer,  # Получатель: разработчик
            "message": """Реализовать программу, которая получает данные из временой таблицы,
удаляет дубликаты и записывает результат в историческую таблицу""",  # Задача: написать программу для обработки таблиц
            "max_turns": 1,  # Один обмен сообщениями (запрос-ответ)
            "summary_method": "last_msg",  # Суммаризация: использовать последнее сообщение
        },
        {
            "recipient": qa_engineer,  # Получатель: тестировщик
            "message": """Вот мои unit tests""",  # Сообщение: запрос на написание тестов (контекст берётся из предыдущего чата)
            "max_turns": 1,  # Один обмен
            "summary_method": "last_msg",  # Суммаризация: последнее сообщение
        },
        {
            "recipient": devops_engineer,  # Получатель: DevOps-инженер
            "message": """Вот моя реализация CI/CD""",  # Сообщение: запрос на настройку CI/CD (контекст из предыдущих чатов)
            "max_turns": 1,  # Один обмен
            "summary_method": "last_msg",  # Суммаризация: последнее сообщение
        },
    ]
)

# Вывод результатов чатов
print("\n\nFirst Chat Summary (software_engineer):\n", chat_results[0].summary)  # Вывод суммаризации первого чата (код от разработчика)
print("\n\nSecond Chat Summary (qa_engineer):\n", chat_results[1].summary)  # Вывод суммаризации второго чата (юнит-тесты)
print("\n\nThird Chat Summary (devops_engineer):\n", chat_results[2].summary)  # Вывод суммаризации третьего чата (CI/CD)


********************************************************************************
Starting a new chat....

********************************************************************************
product_manager (to software_engineer):

Реализовать программу, которая получает данные из временой таблицы,
удаляет дубликаты и записывает результат в историческую таблицу

--------------------------------------------------------------------------------
software_engineer (to product_manager):


Для реализации этой задачи мы можем использовать следующий код:
```
import pandas as pd

# Создаем временную таблицу с данными
data = {'name': ['John', 'Mary', 'John', 'Jane'],
        'age': [25, 30, 35, 40]}
df = pd.DataFrame(data)

# Удаляем дубликаты из временной таблицы
df_unique = df.drop_duplicates()

# Записываем результат в историческую таблицу
df_unique.to_sql('history', con=engine, if_exists='append')
```
В этом коде мы используем библиотеку pandas для работы с данными и создаем временную таблицу `d

In [4]:
import re  # Для работы с регулярными выражениями

# Конфигурация моделей для разных задач
# config_list: Основная конфигурация для небольших моделей с низким потреблением токенов
config_list = [
    {
        "model": "codellama:7b",
        # "model": "llama3.2:3b",  # Модель LLaMA 3.2 с 3B параметров
        "api_type": "ollama",  # Используем Ollama для локального хостинга модели
        "client_host": "http://localhost:11434",  # Адрес сервера Ollama
        "num_predict": 1000,  # Максимальное количество токенов в ответе
        "temperature": 0.3,  # Низкая температура для предсказуемых ответов
        "top_p": 0.95,  # Топ-95% вероятных токенов для генерации
        "stream": False  # Отключение потоковой генерации
    }
]

In [5]:
# config_list_think: Конфигурация для более сложных задач, требующих анализа
config_list_think = [
    {
        "model": "codellama:7b",
        # "model": "qwen3:8b",  # Модель Qwen 3 с 8B параметров для глубокого анализа
        "api_type": "ollama",
        "client_host": "http://localhost:11434",
        "num_predict": 1500,  # Увеличенный лимит токенов для длинных ответов
        "temperature": 0.3,
        "top_p": 0.95,
        "stream": False
    }
]

# config_list_code: Конфигурация для задач, связанных с кодом
config_list_code = [
    {
        "model": "codellama:7b",  # Модель CodeLLaMA для генерации и анализа кода
        "api_type": "ollama",
        "client_host": "http://localhost:11434",
        "num_predict": 1500,
        "temperature": 0.3,
        "top_p": 0.95,
        "stream": False
    }
]

In [6]:
# Определение агентов для выполнения задач проекта
# Каждый агент — это ConversableAgent с уникальной ролью и конфигурацией LLM
project_manager = autogen.ConversableAgent(
    name="project_manager",  # Имя агента
    system_message="""Менеджер продукта. Координируй команду инженеров. Запрашивай план у planner, оценивай его (1–10). Если оценка < 5, отправляй на доработку. Контролируй выполнение этапов, оценивай результаты и давай обратную связь.""",
    # Инструкции для агента
    llm_config={"config_list": config_list_think},  # Используем модель для анализа
    human_input_mode="TERMINATE"  # Требуется вмешательство человека для завершения
)

business_analyst = autogen.ConversableAgent(
    name="business_analyst",
    system_message="""Бизнес-аналитик. Собирай и анализируй данные о рынке. Структурируй выводы в JSON. Завершай словом DONE.""",
    llm_config={"config_list": config_list_think},
    human_input_mode="NEVER"  # Работает автономно
)

rnd_engineer = autogen.ConversableAgent(
    name="rnd_engineer",
    system_message="""R&D инженер. Разрабатывай прототипы и алгоритмы для голосового помощника. Завершай словом DONE.""",
    llm_config={"config_list": config_list_think},
    human_input_mode="NEVER"
)

ml_engineer = autogen.ConversableAgent(
    name="ml_engineer",
    system_message="""ML-инженер. Создавай и тестируй модели для голосового помощника. Завершай словом DONE.""",
    llm_config={"config_list": config_list_code},  # Специализированная модель для кода
    human_input_mode="NEVER"
)

In [7]:
data_engineer = autogen.ConversableAgent(
    name="data_engineer",
    system_message="""Дата-инженер. Настрой инфраструктуру для данных голосового помощника. Завершай словом DONE.""",
    llm_config={"config_list": config_list_code},
    human_input_mode="NEVER"
)

ab_testing_specialist = autogen.ConversableAgent(
    name="ab_testing_specialist",
    system_message="""Специалист по A/B тестированию. Планируй и проводи тесты для оценки голосового помощника. Завершай словом DONE.""",
    llm_config={"config_list": config_list_think},
    human_input_mode="NEVER"
)

planner = autogen.ConversableAgent(
    name="planner",
    system_message="""Планировщик. Создай план выполнения задачи в формате JSON: {'stages': [{'name': ..., 'agent': ..., 'task': ...}, ...]}. Пересмотри план на основе отзывов project_manager и critic. Завершай словом DONE.""",
    llm_config={"config_list": config_list},  # Лёгкая модель для планирования
    human_input_mode="NEVER"
)

critic = autogen.ConversableAgent(
    name="critic",
    system_message="""Критик. Проверяй план и результаты агентов. Оцени прибыльность для банка (1–10). Завершай словом DONE.""",
    llm_config={"config_list": config_list_think},
    human_input_mode="NEVER"
)

cto = autogen.ConversableAgent(
    name="cto",
    system_message="""Технический директор банка. Оцени реализацию проекта, указывая сильные и слабые стороны. Завершай словом DONE.""",
    llm_config={"config_list": config_list_think},
    human_input_mode="NEVER"
)

In [8]:
# Назначение описаний для агентов (используется в чате для идентификации ролей)
project_manager.description = "Координирует команду инженеров."
business_analyst.description = "Анализирует рынок."
rnd_engineer.description = "Разрабатывает прототипы."
ml_engineer.description = "Создаёт ML-модели."
data_engineer.description = "Настраивает инфраструктуру данных."
ab_testing_specialist.description = "Проводит A/B тестирование."
planner.description = "Создаёт план проекта."
critic.description = "Оценивает план и результаты."

# Ограничения переходов между агентами в групповом чате
# Ключи — агенты, значения — списки агентов, которым можно передать слово
allowed_transitions = {
    project_manager: [business_analyst, critic],  # Менеджер передаёт аналитику или критику
    business_analyst: [rnd_engineer, ab_testing_specialist, project_manager],  # Аналитик передаёт R&D или тестировщику
    rnd_engineer: [ml_engineer, data_engineer],  # R&D передаёт ML или дата-инженеру
    ml_engineer: [data_engineer, business_analyst],  # ML передаёт дата-инженеру или аналитику
    data_engineer: [ml_engineer, business_analyst],  # Дата-инженер возвращает ML или аналитику
    ab_testing_specialist: [business_analyst, rnd_engineer],  # Тестировщик передаёт аналитику или R&D
    planner: [critic],  # Планировщик передаёт критику
    critic: [planner, project_manager]  # Критик возвращает планировщику или менеджеру
}

In [9]:
# Список агентов для группового чата (без cto, так как он участвует только во вложенном чате)
agents = [
    project_manager,
    business_analyst,
    rnd_engineer,
    ml_engineer,
    data_engineer,
    ab_testing_specialist,
    planner,
    critic
]

# Создание группового чата
group_chat = autogen.GroupChat(
    agents=agents,  # Список участвующих агентов
    allowed_or_disallowed_speaker_transitions=allowed_transitions,  # Ограничения переходов
    speaker_transitions_type="allowed",  # Тип переходов: только разрешённые
    messages=[],  # Начальный список сообщений пуст
    max_round=10,  # Максимум 10 раундов обсуждения
    send_introductions=True,  # Агенты представляются в начале
    select_speaker_auto_verbose=True  # Логирование выбора спикера для отладки
)

In [10]:


# Менеджер группового чата
group_chat_manager = autogen.GroupChatManager(
    groupchat=group_chat,  # Ссылка на объект группового чата
    llm_config={"config_list": config_list}  # Конфигурация модели для менеджера
)

# Оркестратор для управления вложенными чатами
orchestrator_agent = autogen.ConversableAgent(
    name="OrchestratorAgent",  # Имя агента-оркестратора
    llm_config=False,  # Без модели, так как оркестратор только управляет
    human_input_mode="NEVER",  # Полностью автономный
    code_execution_config=False  # Без выполнения кода
)

task = """Мы хотим разработать голосовых помощников, которые будут работать в нашем банковском мобильном приложении.
Сначала нужно проанализировать рынок. Также нужно будет провести A/B тестирование."""

# Вложенные чаты для последовательного выполнения задач
nested_chats = [
    {
        "recipient": planner,  # Первый чат с планировщиком
        "message": f"Создай план для задачи: {task}",  # Запрос на создание плана
        "summary_method": "reflection_with_llm",  # Суммаризация с помощью LLM
        "summary_prompt": "Суммируй план в формате JSON.",  # Инструкция для суммаризации
        "max_turns": 1  # Один обмен сообщениями
    },
    {
        "recipient": group_chat_manager,  # Второй чат с групповым менеджером
        "message": "Реализуй проект по плану: {summary}",  # Запрос на реализацию плана
        "summary_method": "reflection_with_llm",
        "max_round": 1  # Ошибка: должно быть max_turns (исправлено ниже в подсказке)
    },
    {
        "recipient": cto,  # Третий чат с CTO
        "message": "Оцени реализацию проекта",  # Запрос на оценку
        "summary_method": "reflection_with_llm",
        "max_turns": 1  # Один обмен
    }
]

# Регистрация вложенных чатов в оркестраторе
orchestrator_agent.register_nested_chats(
    nested_chats,  # Список вложенных чатов
    trigger=lambda sender: sender not in [group_chat_manager, cto]  # Условие активации: отправитель не из исключённых
)

# Функция для очистки ответа
def clean_summary(reply):
    """
    Очищает ответ от исходного запроса и лишних символов.

    Args:
        reply: Ответ от агента (словарь или строка).

    Returns:
        str: Очищенный текст ответа.
    """
    # Извлекаем строку из словаря или используем reply как строку
    if isinstance(reply, dict):
        text = reply.get("content", "")
    else:
        text = str(reply)
    # Удаляем исходный запрос из ответа с помощью регулярного выражения
    return re.sub(r"Мы хотим разработать голосовых помощников.*?\n", "", text, flags=re.DOTALL).strip()

# Запуск задачи
# Генерация ответа оркестратором
reply = orchestrator_agent.generate_reply(
    messages=[{"role": "user", "content": task}]  # Входное сообщение с задачей
)

# Вывод очищенного ответа
print(f"\n\nCleaned Reply: \n{clean_summary(reply)}")


********************************************************************************
Starting a new chat....

********************************************************************************
OrchestratorAgent (to planner):

Создай план для задачи: Мы хотим разработать голосовых помощников, которые будут работать в нашем банковском мобильном приложении. 
Сначала нужно проанализировать рынок. Также нужно будет провести A/B тестирование.

--------------------------------------------------------------------------------
planner (to OrchestratorAgent):


{
"stages": [
{"name": "Market Analysis", "agent": "project_manager", "task": "Conduct market research to identify potential customers and their needs"},
{"name": "A/B Testing", "agent": "critic", "task": "Design and conduct A/B testing to determine the best approach for our voice assistants"}
]
}

DONE.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (1fe88a0f-a203-457b-b461-5225dd500